### Select your source mapping file
Mapping file must be excel file that conforms to the mapping standard

In [ ]:
import pandas as pd
import fedelemflowlist
from fedelemflowlist.globals import flowmappingpath, flowmapping_fields
import xlrd
from tkinter import filedialog

In [ ]:
file = filedialog.askopenfilename()

In [ ]:
mapping = pd.read_excel(file)
mapping_length = len(mapping)
print(mapping_length)
all_flows = fedelemflowlist.get_flows()
all_flows = all_flows[['Flowable', 'Context', 'Flow UUID', 'Unit']]
mapping_w_flowinfo = pd.merge(mapping, all_flows,
            left_on=['TargetFlowName', 'TargetFlowContext', 'TargetUnit'],
            right_on=['Flowable', 'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.drop(columns=['TargetFlowUUID', 'Flowable',
                                        'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.rename(columns={'Flow UUID': 'TargetFlowUUID'})
mapping_merged_len = len(mapping_w_flowinfo)
if mapping_length > mapping_merged_len:
    print("Not all flows were mapped to flows in the list")

flowmapping_order = list(flowmapping_fields.keys())
mapping_w_flowinfo = mapping_w_flowinfo[flowmapping_order]
print(mapping_w_flowinfo)

In [ ]:
directory = filedialog.askdirectory()

In [ ]:
mapping_flow_uuids = pd.DataFrame(pd.unique(mapping_w_flowinfo['TargetFlowUUID']),columns=["Flow UUID"])

#Get all flows
all_flows = fedelemflowlist.get_flows()
#Subset all flows to get just those used in selected mapping
flows_used_in_mapping =  pd.merge(all_flows,mapping_flow_uuids)

#Now write out flows and mappings
export_name = 'flows_w_mappings.zip'
fedelemflowlist.write_jsonld(flows_used_in_mapping,directory+'/'+export_name,mapping_w_flowinfo)
print("mapping file created as json")